In [ ]:
"""
Adapted from salesforce@LAVIS and Vision-CAIR@MiniGPT-4. Below is the original copyright:
 Copyright (c) 2022, salesforce.com, inc.
 All rights reserved.
 SPDX-License-Identifier: BSD-3-Clause
 For full license text, see the LICENSE_Lavis file in the repo root or https://opensource.org/licenses/BSD-3-Clause
"""

import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn

import video_llama.tasks as tasks
from video_llama.common.config import Config
from video_llama.common.dist_utils import get_rank, init_distributed_mode
from video_llama.common.logger import setup_logger
from video_llama.common.optims import (
    LinearWarmupCosineLRScheduler,
    LinearWarmupStepLRScheduler,
)
from video_llama.common.registry import registry
from video_llama.common.utils import now

# imports modules for registration
from video_llama.datasets.builders import *
from video_llama.models import *
from video_llama.processors import *
from video_llama.runners import *
from video_llama.tasks import *

import yaml ## ADD


def parse_args():
    parser = argparse.ArgumentParser(description="Training")

    parser.add_argument("--cfg-path", required=True, help="path to configuration file.")
    parser.add_argument(
        "--options",
        nargs="+",
        help="override some settings in the used config, the key-value pair "
        "in xxx=yyy format will be merged into config file (deprecate), "
        "change to --cfg-options instead.",
    )

    args = parser.parse_args()
    # if 'LOCAL_RANK' not in os.environ:
    #     os.environ['LOCAL_RANK'] = str(args.local_rank)

    return args


def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True


def get_runner_class(cfg):
    """
    Get runner class from config. Default to epoch-based runner.
    """
    runner_cls = registry.get_runner_class(cfg.run_cfg.get("runner", "runner_base"))

    return runner_cls


def setup_runner(cfg):
    # set before init_distributed_mode() to ensure the same job_id shared across all ranks.
    job_id = now()

    init_distributed_mode(cfg.run_cfg)

    # setup_seeds(cfg.run_cfg.seed + get_rank())

    # set after init_distributed_mode() to only log on master.
    setup_logger()

    cfg.pretty_print()

    task = tasks.setup_task(cfg)
    datasets = task.build_datasets(cfg)

    runner_cls = get_runner_class(cfg)
    runner = runner_cls(
        cfg=cfg, job_id=job_id, task=task, model=None, datasets=datasets
    )

    return runner

args = argparse.Namespace()

# Set attributes manually
args.cfg_path = "./train_configs/drive_finetune.yaml"
args.options = []

cfg = Config(args)

# # Update the configuration with the loaded dictionary
# for key, value in cfg_dict.items():
#     setattr(cfg, key, value)
# # Setup the runner
# setup_seeds(cfg.run_cfg.seed + get_rank())

job_id = now()

init_distributed_mode(cfg.run_cfg)

# setup_seeds(cfg.run_cfg.seed + get_rank())

# set after init_distributed_mode() to only log on master.
setup_logger()

cfg.pretty_print()

task = tasks.setup_task(cfg)
datasets = task.build_datasets(cfg)

runner_cls = get_runner_class(cfg)
runner = runner_cls(
    cfg=cfg, job_id=job_id, task=task, model=None, datasets=datasets
)

# Train using the runner
# runner.train()



In [5]:
datasets['bdd_instruct']

{'train': <video_llama.datasets.datasets.bdd_instruct_dataset.BDD_Instruct_Dataset at 0x29d0dd600>}

In [3]:
for index, data in datasets['webvid_instruct']['train']:
    print(f"Sample {index + 1}:")
    print(f"Keys in sample: {list(data.keys())}")
    # Print out specific data from the sample
    print(f"Video path: {data['video_path']}")
    print(f"Target label: {data['target_label']}")
    # Add more data fields as needed
    print("-" * 30)

UnboundLocalError: local variable 'video_path' referenced before assignment